In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import numpy as np
import xarray as xr
import netCDF4
import matplotlib.colors as mcolors
import matplotlib.image as mpimg
from IPython.display import Image, display
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature
from modules.plotter import make_cmap
import matplotlib.gridspec as gridspec
%matplotlib inline

### GPM Precip Accumulation

#### Import GPM Precipitation Data

In [2]:
def preprocess(ds):
    '''keep only the selected lats and lons'''
    return ds.sel(lat=slice(32,42), lon=slice(-125,-115))

filename_pattern =  '/home/sbarc/students/nash/data/GPM/imerg_2March19/3B*' 
ds = xr.open_mfdataset(filename_pattern, engine='netcdf4', concat_dim='time', group='Grid',
                      preprocess=preprocess)
print('ds size in GB {:0.2f}\n'.format(ds.nbytes / 1e9))
# ds.info()

precip = ds.precipitationCal.transpose('time', 'lat', 'lon')
precip = precip.load()
precip_final = precip.sum('time')/2.0

ds size in GB 0.02



In [3]:
## Santa Barbara Rain Information
SB_precip = precip.sel(lat=[34.4208], lon=[-119.6982], method='nearest')

# Total accumulated over SB
SB_precip_total = SB_precip.sum('time')/2.0
print('Total accumulated in SB:', SB_precip_total.values, 'mm')
# Max rain rate and time
SB_max = SB_precip.where(SB_precip==SB_precip.max(), drop=True).squeeze()
print('Max rain rate:', SB_max.values, 'mm/hour at', SB_max.time.values)


Total accumulated in SB: [[13.915319]] mm
Max rain rate: 2.9003837 mm/hour at 2019-03-02 07:30:00


In [4]:
## Goleta Fire Station Rain Information
gfire_precip = precip.sel(lat=[34.25], lon=[-119.55], method='nearest')
# Total accumulated over SB
gfire_precip_total = gfire_precip.sum('time')/2.0
print('Total accumulated at Goleta Fire Station:', gfire_precip_total.values, 'mm')
# Max rain rate and time
gfire_max = gfire_precip.where(gfire_precip==gfire_precip.max(), drop=True).squeeze()
print('Max rain rate:', gfire_max.values, 'mm/hour at', gfire_max.time.values)

Total accumulated at Goleta Fire Station: [[17.487797]] mm
Max rain rate: 6.698513 mm/hour at 2019-03-02 09:30:00
